In [12]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try: # Currently, memory growth needs to be the same across GPUs 
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e: # Memory growth must be set before GPUs have been initialized print(e)
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from minigooglenet import MiniGoogLeNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
print("[INFO] accessing CIFAR-10 data...")
((trainX, trainY),(testX,testY))=cifar10.load_data()

trainX=trainX.astype("float")/255.0
testX=testX.astype("float")/255.0

print(trainX.shape)
print(testX.shape)

# convert the labels from integers to vectors
lb=LabelBinarizer()
trainY=lb.fit_transform(trainY)
testY=lb.transform(testY)

print(trainY.shape)
print(testY.shape)

# initialize the label names for the CIFAR-10 dataset
labelNames=["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

[INFO] accessing CIFAR-10 data...
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [21]:
# constuct the image generator for data augmentation
train_generator=ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
augment_size=10000
randidx=np.random.randint(trainX.shape[0], size=augment_size)
x_augmented=trainX[randidx].copy()
y_augmented=trainY[randidx].copy()

x_augmented=train_generator.flow(x_augmented,np.zeros(augment_size),batch_size=augment_size,shuffle=True)

trainX=np.concatenate((trainX,x_augmented))
trainY=np.concatenate((trainY,y_augmented))

print(trainX.shape)

MemoryError: Unable to allocate 117. MiB for an array with shape (10000, 32, 32, 3) and data type float32

In [ ]:
# initialize the optimizer and model
epoch_param=10
print("[INFO] compiling model...")
INIT_LR=0.01
opt=SGD(lr=INIT_LR,momentum=0.9)
model=MiniGoogleNet.build(widtgh=32,height=32,depth=3,classes=10)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H=model.fit(trainX,trainY,validation_data=(testX,testY),batch_size=64,epochs=epoch_param,verbose=0)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions=model.predict(testX,batch_size=64)
cr=classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=[str(x) for x in range(10)])
print(cr)

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,epoch_param),H.history["loss"],label="train_loss")
plt.plot(np.arange(0,epoch_param),H.history["val_loss"],label="val_loss")
plt.plot(np.arange(0,epoch_param),H.history["accuracy"],label="train_acc")
plt.plot(np.arange(0,epoch_param),H.history["val_accuracy"],label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [22]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}